# List of Postal Codes in Toronto Exercise 1 - Segmenting and Clustering Neighbourhoods

## Set up section

#### This first batch of code is to load all the relevent directories etc


In [158]:
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


!conda install -c conda-forge folium=0.5.0 --yes 
import folium 
from urllib import request
import requests
import urllib.request
import time
from bs4 import BeautifulSoup



Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [159]:
import folium # map rendering library
from urllib import request
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import bs4 as bs


print('Libraries imported.')

Libraries imported.


### This section will scrape the website for the basic details
#### 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.html
#### I used Python BeautifulSoup and Python lxml

In [160]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
rawpage = request.urlopen(url)

In [161]:
def scrape_table_bs4(cname,cols):
    page = urllib.request.urlopen(url).read()
    soup = bs.BeautifulSoup(page,'lxml')
    table = soup.find('table',class_=cname)
    header = [head.findAll(text=True)[0].strip() for head in table.find_all("th")]
    data = [[td.findAll(text=True)[0].strip() for td in tr.find_all("td")] for tr in table.find_all("tr")]
    data = [row for row in data if len(row) == cols]
# This is to store the data temporarily
    temp_df = pd.DataFrame(data, columns=header)
    return temp_df

### This is to parse the data uisng xpath

In [162]:
def scrape_table_lxml(XPATH,cols):
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    table_content = doc.xpath(XPATH)
    for table in table_content: headers = [th.text_content().strip() for th in table.xpath('//th')]
    headers = headers[0:3]
    data = [[td.text_content().strip() for td in tr.xpath('td')] for tr in table.xpath('//tbody/tr')]
    data = [row for row in data if len(row) == cols]
    temp_df = pd.DataFrame(data, columns=headers)
    return temp_df        

### This is to test the work in Beautiful Soup

In [163]:
raw_Postcodes = scrape_table_bs4("wikitable",3)

### This is to test the work in LXML

In [164]:

print ("Postcodes")
print(raw_Postcodes.info(verbose = True))

Postcodes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 3 columns):
Postcode         287 non-null object
Borough          287 non-null object
Neighbourhood    287 non-null object
dtypes: object(3)
memory usage: 6.8+ KB
None


# Assumptions

#### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

#### I will only process the cells that have an assigned borough. I will ignore cells with a borough that is Not assigned.

#### Where more than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma.

#### If a cell has a borough but a Not assigned neighborhood, then I will make the neighborhood will be the same as the borough. For example the 9th cell in 

In [165]:
Postcodes = raw_Postcodes[~raw_Postcodes['Borough'].isin(['Not assigned'])]
                          
Postcodes=Postcodes.sort_values(by=['Postcode', 'Borough', 'Neighbourhood'], ascending =[1,1,1]).reset_index(drop=True)

In [166]:
Postcodes.loc[Postcodes['Neighbourhood'] == 'Not assigned', ['Neighbourhood']]=Postcodes['Borough']

check_unassigned_post_state_sample = Postcodes.loc[Postcodes['Borough'] =='Queen\'s Park']


In [167]:
Postcodes = Postcodes.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

### List if Postal Codes in Toronto Canada (starting M...)

In [168]:
Postcodes

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [169]:
Postcodes.shape

(103, 3)

# Second Part of Assignment
## Get Coords of each Neighbourhood

In order to utilize the Foursquare location data, I will get the latitude and the longitude coordinates of each neighborhood.
 Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

I chose to use the provided csv file  becuase of   - http://cocl.us/Geospatial_data

In [170]:
lat_longcsv = 'http://cocl.us/Geospatial_data'
!wget -q -o 'Geospatial_coordinates.csv' lat_longcsv
geopostcode_data=pd.read_csv(lat_longcsv).set_index('Postal Code')
geopostcode_data.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [171]:
Postcodes.to_csv('postcode1_df.csv',index=False)

postcode_csv = 'postcode1_df.csv'

postcodes1 = pd.read_csv(postcode_csv).set_index('Postcode')
postcodes1.rename_axis('Postal Code', axis = 'index', inplace = True)
postcodes1.head()


,Borough,Neighbourhood
Postal Code,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


#### Combine the two dataframes using postal codes as key

In [172]:
Combined_data = postcodes1.join( geopostcode_data)
Combined_data.head()

,Borough,Neighbourhood,Latitude,Longitude
Postal Code,,,,
M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [173]:
Combined_data.shape

(103, 4)

# Final Part of the Assignment
## Exploring and clustering the neighbourhoods in Toronto

#### To explore the neighbourhoos of selected cities I will use the Foursquare API.  

#### Firstly I will load all the libraries


In [174]:
!conda install -c conda-forge geopy --yes        
!conda install -c conda-forge folium=0.5.0 --yes 

import numpy as np 
import pandas as pd 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 


from geopy.geocoders import Nominatim 
GeoLocator = Nominatim(user_agent='My-IBMNotebook')

import requests 
from pandas.io.json import json_normalize 


import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans


import folium 

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


### Use geophy to get the lat/long values of Toronto Canada

In [175]:
address = 'Toronto, Ontario Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto Canada are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Toronto Canada are 43.653963, -79.387207.


### Create a map of Toronto with the cities superimposed

In [176]:

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)


for lat, lng, borough, neighborhood in zip(Combined_data['Latitude'], Combined_data['Longitude'], Combined_data['Borough'], Combined_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)

In [177]:
map_toronto

###  Assumption

#### For the purpose of te exercise I will  work with only boroughs that contain the word Toronto and then replicate the same analysis that I did with the New York City data.

In [178]:
toronto_data = Combined_data[Combined_data['Borough'].str.contains("Toronto")].reset_index(drop=True)
print(toronto_data.shape)
toronto_data.head()

(38, 4)


,Borough,Neighbourhood,Latitude,Longitude
0,East Toronto,The Beaches,43.676357,-79.293031
1,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188
2,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,East Toronto,Studio District,43.659526,-79.340923
4,Central Toronto,Lawrence Park,43.728020,-79.388790


#### I will now recreate the map with the markers on it for the neighourhoods

In [179]:

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)


for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Now using the Foursquare API to explore and segment neighborhoods



In [180]:
CLIENT_ID = 'DWE403I3DYSRFXV4VDIAQOSUD1IMFKWNV4LMVNQWSR5CZMDV' # your Foursquare ID
CLIENT_SECRET = 'UND1K2GR13ZF5ZUYY45MAQINZRAGY4IJ2EXBINYW0FAOPGGI' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DWE403I3DYSRFXV4VDIAQOSUD1IMFKWNV4LMVNQWSR5CZMDV
CLIENT_SECRET:UND1K2GR13ZF5ZUYY45MAQINZRAGY4IJ2EXBINYW0FAOPGGI


### To explore the neighbourhoods in Toronto

#### I will use the same query as for the NY exercise
https://api.foursquare.com/v2/venues/search?
client_id=CLIENT_ID&client_secret=CLIENT_SECRET&ll=LATITUDE,LONGITUDE&v=VERSION&query=QUERY&radius=RADIUS&limit=LIMIT

In [181]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
      
    
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
     
        answer = requests.get(url).json()
        results = answer['response']['groups'][0]['items']
        
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [182]:
Combined_data = toronto_data
toronto_venues = getNearbyVenues(names=Combined_data['Neighbourhood'],
                                   latitudes=Combined_data['Latitude'],
                                   longitudes=Combined_data['Longitude'])

The Beaches
Riverdale, The Danforth West
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
North Midtown, The Annex, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
Bathurst Quay, CN Tower, Harbourfront West, Island airport, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

In [183]:
Combined_data.shape

(38, 4)

In [184]:
toronto_venues.head()

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"Riverdale, The Danforth West",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [185]:
toronto_venues.shape

(836, 7)

#### Noting that 836 venues have been returned, lets check to see how many venues are in each neighbourhood

In [186]:
toronto_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",30,30,30,30,30,30
"Bathurst Quay, CN Tower, Harbourfront West, Island airport, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16
Berczy Park,30,30,30,30,30,30
"Brockton, Exhibition Place, Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,18,18,18,18,18,18
"Cabbagetown, St. James Town",30,30,30,30,30,30
Central Bay Street,30,30,30,30,30,30
"Chinatown, Grange Park, Kensington Market",30,30,30,30,30,30
Christie,17,17,17,17,17,17


#### Checking on unique categories in each area

In [187]:
print('{} unique venue categories have been found.'.format(len(toronto_venues['Venue Category'].unique())))

184 unique venue categories have been found.


## Analysing each neighbourhood
### Using One Hot  encoding
#### One hot encoding is a process by which categorical variables are converted into a form that could be provided to ML algorithms to do a better job in prediction
#### then sort out the presentation of the data

In [188]:
venues_oh = pd.get_dummies(toronto_venues['Venue Category'])


venues_oh['Neighbourhood'] = toronto_venues['Neighbourhood'] 


fixed_columns = [venues_oh.columns[-1]] + list(venues_oh.columns[:-1])
venues_oh =venues_oh[fixed_columns]

venues_oh.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Dog Run,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

# just confirm that no venues have been dropped (check figure is 836)

In [189]:
venues_oh.shape

(836, 185)

#### Lets work out hte aaverage number of the different types of venues for each neighbourhood to see which ones suit me the best

In [190]:
neighbourhoods_grouped = venues_oh.groupby('Neighbourhood').mean().reset_index()

In [191]:
neighbourhoods_grouped

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Dog Run,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Adelaide, King, Richmond",0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.033333,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.0000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000

#### check for new size in case items get dropped in the future

In [192]:
neighbourhoods_grouped.shape

(38, 185)

#### convert to a panda's dataframe for easier use later on

In [193]:
num_top = 5
for neigh in neighbourhoods_grouped['Neighbourhood']:
    print(""+neigh+"")
    temp = neighbourhoods_grouped[neighbourhoods_grouped['Neighbourhood'] == neigh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top))
    print('\n')

Adelaide, King, Richmond
              venue  freq
0              Café  0.10
1        Steakhouse  0.10
2             Hotel  0.07
3  Sushi Restaurant  0.07
4  Asian Restaurant  0.07


Bathurst Quay, CN Tower, Harbourfront West, Island airport, King and Spadina, Railway Lands, South Niagara
              venue  freq
0   Airport Service  0.19
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3           Airport  0.06
4               Bar  0.06


Berczy Park
                venue  freq
0  Seafood Restaurant  0.07
1        Cocktail Bar  0.07
2         Coffee Shop  0.07
3            Beer Bar  0.07
4      Farmers Market  0.07


Brockton, Exhibition Place, Parkdale Village
            venue  freq
0     Coffee Shop  0.09
1            Café  0.09
2  Breakfast Spot  0.09
3     Yoga Studio  0.05
4          Bakery  0.05


Business Reply Mail Processing Centre 969 Eastern
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.06
2       Garden Center  0.06
3          Comic 

#### intersting but not very easy to understand
#### lets put them in descending order - so I can see the top 20

In [194]:
def return_most_common_venues(row, num_top):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top]

In [195]:
num_top = 20

indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arange(num_top):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


nvs = pd.DataFrame(columns=columns)
nvs['Neighbourhood'] = neighbourhoods_grouped['Neighbourhood']

for ind in np.arange(neighbourhoods_grouped.shape[0]):
    nvs.iloc[ind, 1:] = return_most_common_venues(neighbourhoods_grouped.iloc[ind, :], num_top)

nvs.shape

(38, 21)

In [196]:
nvs.head(20)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,"Adelaide, King, Richmond",Café,Steakhouse,Hotel,Asian Restaurant,Sushi Restaurant,Monument / Landmark,Pizza Place,Concert Hall,Plaza,Neighborhood,Lounge,Seafood Restaurant,Opera House,Smoke Shop,Speakeasy,Bar,Noodle House,Gym / Fitness Center,Coffee Shop,Vegetarian / Vegan Restaurant
1,"Bathurst Quay, CN Tower, Harbourfront West, Is...",Airport Service,Airport Lounge,Airport Terminal,Airport,Bar,Coffee Shop,Plane,Sculpture Garden,Boat or Ferry,Airport Food Court,Airport Gate,Harbor / Marina,Gift Shop,Diner,Falafel Restaurant,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store
2,Berczy Park,Seafood Restaurant,Farmers Market,Coffee Shop,Cocktail Bar,Beer Bar,Bistro,Fountain,Breakfast Spot,Jazz Club,Concert Hall,Italian Restaurant,Basketball Stadium,Fish Market,Steakhouse,Bakery,Cheese Shop,Liquor Store,Clothing Store,Tea Room,French Restaurant
3,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Breakfast Spot,Café,Performing Arts Venue,Burrito Place,Restaurant,Climbing Gym,Pet Store,Nightclub,Bar,Convenience Store,Italian Restaurant,Intersection,Furniture / Home Store,Gym,Grocery Store,Stadium,Yoga Studio,Bakery,Flea Market
4,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Brewery,Spa,Park,Farmers Market,Fast Food Restaurant,Burrito Place,Butcher,Restaurant,Skate Park,Smoke Shop,Comic Shop,Garden,Pizza Place,Garden Center,College Gym,Comfort Food Restaurant,Discount Store
5,"Cabbagetown, St. James Town",Café,Coffee Shop,Restaurant,Italian Restaurant,Pet Store,Pub,Liquor Store,Butcher,Jewelry Store,Japanese Restaurant,Diner,Beer Store,Indian Restaurant,Caribbean Restaurant,Bank,Bakery,Market,Sandwich Place,Park,Taiwanese Restaurant
6,Central Bay Street,Coffee Shop,Italian Restaurant,Chinese Restaurant,Café,Bubble Tea Shop,Spa,Gastropub,Ice Cream Shop,Sandwich Place,Ramen Restaurant,Japanese Restaurant,Miscellaneous Shop,Bar,Poke Place,Seafood Restaurant,Modern European Restaurant,Sushi Restaurant,Tea Room,Comic Shop,Park
7,"Chinatown, Grange Park, Kensington Market",Café,Vietnamese Restaurant,Comfort Food Restaurant,Mexican Restaurant,Bakery,Farmers Market,Dessert Shop,Bar,Belgian Restaurant,Caribbean Restaurant,Cheese Shop,Noodle House,Dumpling Restaurant,Grocery Store,Arts & Crafts Store,Fish Market,Gaming Cafe,Cocktail Bar,Thrift / Vintage Store,Coffee Shop
8,Christie,Café,Grocery Store,Park,Diner,Baby Store,Candy Store,Athletics & Sports,Restaurant,Coffee Shop,Italian Restaurant,Nightclub,Convenience Store,Discount Store,Dumpling Restaurant,Dessert Shop,Eastern European Restaurant,Dog Run,Dance Studio,Deli / Bodega,Falafel Restaurant
9,Church and Wellesley,Gay Bar,Gastropub,Diner,Indian Restaurant,Pub,Ramen Restaurant,Dessert Shop,Japanese Restaurant,Ethiopian Restaurant,Bookstore,Dance Studio,Juice Bar,Breakfast Spot,Restaurant,Bubble Tea Shop,Burger Joint,Steakhouse,Hobby Shop,Salon / Barbershop,Mexican Restaurant


#### Looks like Adelaide, King, Richmond are good areas for me.
#### Lets see what other neighbourhoods are in that area.

## Clustering the Neighbourhoods
### Using the k means technique to see what other areas are inthe same vicinity
#### Given that there are over 300 neighbourhoods - I'll make 20 clusters

In [197]:
kclusters = 20
neighbourhoodclustering = neighbourhoods_grouped.drop('Neighbourhood',1)
kmeans = KMeans(n_clusters=kclusters, random_state = 1).fit(neighbourhoodclustering)
print(kmeans.labels_[0:12])
print(len(kmeans.labels_))

[14  8  9  0  7  0  0  0 12  0  1  0]
38


#### Confirm data set to be used going forward

In [198]:
Combined_data.head()

,Borough,Neighbourhood,Latitude,Longitude
0,East Toronto,The Beaches,43.676357,-79.293031
1,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188
2,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,East Toronto,Studio District,43.659526,-79.340923
4,Central Toronto,Lawrence Park,43.728020,-79.388790


In [199]:
Combined_data.shape

(38, 4)

In [200]:
finalised_data = Combined_data

finalised_data['Cluster Labels'] = kmeans.labels_

finalised_data = finalised_data.join(nvs.set_index('Neighbourhood'), on='Neighbourhood')

finalised_data.head()

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,East Toronto,The Beaches,43.676357,-79.293031,14,Health Food Store,Pub,Trail,Neighborhood,Yoga Studio,Dance Studio,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Diner,Dessert Shop,Deli / Bodega,Creperie,Cuban Restaurant,Farmers Market,Coworking Space,Cosmetics Shop,Convenience Store
1,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188,8,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Yoga Studio,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant,Pub,Coffee Shop,Diner,Dessert Shop,Spa,Juice Bar,Fruit & Vegetable Store,Furniture / Home Store,Grocery Store,Bookstore,Pizza Place,Trail
2,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,9,Park,Sandwich Place,Brewery,Pub,Steakhouse,Italian Restaurant,Sushi Restaurant,Fish & Chips Shop,Burger Joint,Ice Cream Shop,Food & Drink Shop,Movie Theater,Burrito Place,Pizza Place,Light Rail Station,Gym,Pet Store,Liquor Store,Fast Food Restaurant,Dance Studio
3,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,American Restaurant,Italian Restaurant,Bakery,Yoga Studio,Ice Cream Shop,Sandwich Place,Bookstore,Comfort Food Restaurant,Fish Market,Middle Eastern Restaurant,Stationery Store,Bar,Seafood Restaurant,Cheese Shop,Gay Bar,Thai Restaurant,Coworking Space,Neighborhood
4,Central Toronto,Lawrence Park,43.728020,-79.388790,7,Bus Line,Park,Swim School,Yoga Studio,Falafel Restaurant,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Diner,Dessert Shop,Deli / Bodega,Fast Food Restaurant,Dance Studio,Cuban Restaurant,Creperie,Coworking Space,Cosmetics Shop,Convenience Store


#### Lets now see what this looks like on a map so I can see where things are

In [201]:

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, poi, cluster in zip(finalised_data['Latitude'], finalised_data['Longitude'], finalised_data['Neighbourhood'], finalised_data ['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters